In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import pytensor.tensor as pt
import arviz as az
import nutpie

RANDOM_SEED = 99999
rng = np.random.default_rng(RANDOM_SEED)

# A cheatsheet for setting priors for Gaussian process hyperparameters

People fuss a lot of priors in Bayesian models.  Insightful, right?  The thing is though, in a lot of Bayesian models you can set some pretty reasonable priors without thinking that hard about it.  Your sampler will run fine and your results will look alright.  Gaussian processes are not in this category.  Most of the time how you choose your priors will make or break the model and so some serious fussing is required in order to operate these things correctly.  This post is here to help you guide your fussing.  

The most similar resources out there to this post are:
- Micheal Betancourt's post [Robust Gaussian Process Modeling](https://betanalpha.github.io/assets/case_studies/gaussian_processes.html).
- Dan Simpson's [priors for the parameters of Gaussian processes](https://dansblog.netlify.app/posts/2022-09-07-priors5/priors5.html).

These posts are both excellent and give super detailed explanations and the mathematical "whys" behind their reasoning.  Not to mention the fact that it's their original work we're using here.  However I will boldly assert that the problem with them is that they have lots of words and sometimes things with lots of words take a long time to read and understand, especially when GPs are involved.  My goal here will be to try and distill out the practical pieces from these two (and other) resources so this post can be used as more of a cheatsheet for people trying to fit GPs in practice.  If you want to actually learn things, start here and then be sure to go there afterwards. 

#### Disclaimer #1

This note applies to fitting GPs using MCMC in a [PPL](https://en.wikipedia.org/wiki/Probabilistic_programming) like PyMC, Stan, or NumPyro.  If you're trying to fit GPs using optimizers in something like GPFlow, GPJax, or GPyTorch, this advice is still very applicable -- but it's not directly targeted to that scenario.
 
#### Disclaimer #2

This note also only applies to GPs with covariance functions, or kernels, in the Matérn family which all have a lengthscale hyperparameter.  